## Contour plot function

In [1]:
%matplotlib qt5
from biorefineries import lipidcane2g as lc
import biosteam as bst
import numpy as np
import pandas as pd
from biosteam.utils import colors
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from biosteam.utils import colors
from biosteam.plots import (
    plot_contour_2d,
    MetricBar,
    plot_scatter_points,
    plot_contour_single_metric,
    plot_vertical_line,
    rounded_tickmarks_from_data as tickmarks
)
from math import floor, ceil
from biosteam import plots
from biosteam.utils import CABBI_colors
from thermosteam.units_of_measure import format_units
from biosteam.plots.utils import style_axis, style_plot_limits, fill_plot, set_axes_labels
from biosteam import Metric
from warnings import filterwarnings
filterwarnings('ignore', category=bst.utils.DesignWarning)
    
shadecolor = (*colors.neutral.RGBn, 0.20)
linecolor = (*colors.neutral_shade.RGBn, 0.85)
markercolor = (*colors.orange_tint.RGBn, 1)
edgecolor = (*colors.CABBI_black.RGBn, 1)
    
CABBI_colors = (colors.CABBI_yellow.tint(75).RGBn, 
                colors.CABBI_yellow.RGBn,
                colors.CABBI_green.RGBn,
                colors.CABBI_teal_green.shade(60).RGBn)

CABBI_colors_x = (colors.CABBI_blue_light.tint(90).RGBn,
                  colors.CABBI_blue_light.tint(40).RGBn, 
                  colors.CABBI_blue_light.RGBn, 
#                   colors.CABBI_teal.RGBn,
#                   colors.CABBI_teal_green.tint(10).RGBn,
                  colors.CABBI_teal_green.tint(40).shade(15).RGBn,
                  colors.CABBI_teal_green.shade(45).RGBn)

colormaps = [
    LinearSegmentedColormap.from_list('CABBI', CABBI_colors, 25),
    LinearSegmentedColormap.from_list('CABBI', CABBI_colors_x, 25),
    plt.cm.get_cmap('inferno_r'),
    plt.cm.get_cmap('copper_r'),
    plt.cm.get_cmap('bone_r'),
]

def create_contour_plots(load=False, metric_index=0):
    # %% Generate contour data
    x = np.linspace(0.3, 1., 6)
    y = np.linspace(0.02, 0.15, 6)
    X, Y = np.meshgrid(x, y)
    metric = bst.metric
    kg_per_ton = 907.18474
    metrics = lc.all_metric_mockups
    MFPP = metrics[0]
    TCI = metrics[1]
    MFPP.units = format_units(r'$/ton')
    TCI.units = format_units(r'10^6*$')
    file = 'lipid_extraction_analysis'
    configurations = [1, 2]
    agile = [False, True]
    if load:
        data = np.load(file + '.npy')
    else:
        data = lc.evaluate_configurations_across_extraction_efficiency_and_lipid_content(X, Y, 0.70, agile, configurations)
    np.save(file, data)
    data = data[:, :, :, :, metric_index]
    # %% Plot contours
    xlabel = 'Lipid extraction [%]'
    ylabel = "Lipid content [dry wt. %]"
    ylabels = [f'Lipid-cane only\n{ylabel}',
               f'Lipid-cane & lipid-sorghum\n{ylabel}']
    xticks = [30, 40, 60, 80, 100]
    yticks = [2, 5, 10, 15]
    metric = metrics[metric_index]
    
    metric_bar = MetricBar(metric.name, metric.units, colormaps[metric_index], tickmarks(data, 5, 5), 18)
    fig, axes, CSs, CB = plot_contour_single_metric(
        100.*X, 100.*Y, data, xlabel, ylabels, xticks, yticks, metric_bar, 
        fillblack=False, styleaxiskw=dict(xtick0=False), label=True,
        titles=['Configuration I', 'Configuration II'],
    )
    M = len(configurations)
    N = len(agile)
    for i in range(M):
        for j in range(N):
            ax = axes[i, j]
            CS = CSs[i, j]
            plt.sca(ax)
            metric_data = data[:, :, i, j]
            lb = metric_data.min()
            ub = metric_data.max()
            levels = [i for i in CS.levels if lb <= i <= ub]
            CS = plt.contour(100.*X, 100.*Y, data=metric_data, zorder=1e16, linestyles='dashed', linewidths=1.,
                             levels=levels, colors=[linecolor])
            ax.clabel(CS, levels=CS.levels, inline=True, fmt=lambda x: f'{round(x):,}',
                      fontsize=10, colors=[linecolor], zorder=1e16)
            if j == 0:
                lb = 47.5
                ub = 52.5
            else:
                lb = 75
                ub = 80
            baseline = (lb + ub) / 2.
            plt.fill_between([lb, ub], [2], [20], 
                             color=shadecolor,
                             linewidth=1)
            plot_vertical_line(lb, ls='-.',
                               color=linecolor,
                               linewidth=1.0)
            plot_vertical_line(ub, ls='-.',
                               color=linecolor,
                               linewidth=1.0)
            plot_scatter_points([baseline], [10], marker='*', s=125, color=markercolor,
                                edgecolor=edgecolor)


    plt.show()
    return data

## Contour plots

In [2]:
data = create_contour_plots(metric_index=0, load=False)

C:\Users\yrc2\OneDrive\Code\Bioindustrial-Park\BioSTEAM 2.x.x\biorefineries\cornstover\_system.py:498: RuntimeWarning: cannot define water split, only moisture content
  S303 = bst.PressureFilter('S303', slurry, (lignin, ''))
C:\Users\yrc2\OneDrive\Code\Bioindustrial-Park\BioSTEAM 2.x.x\biorefineries\lipidcane2g\_system.py:907: RuntimeWarning: undocked inlet stream s150 from unit S403; s150 is now docked at M404
  MX = bst.Mixer(400, [S303-1, screened_juice])
C:\Users\yrc2\OneDrive\Code\Bioindustrial-Park\BioSTEAM 2.x.x\biorefineries\lipidcane2g\_system.py:907: RuntimeWarning: undocked inlet stream s405 from unit S403; s405 is now docked at M404
  MX = bst.Mixer(400, [S303-1, screened_juice])
C:\Users\yrc2\OneDrive\Code\biosteam\biosteam\_unit.py:658: RuntimeWarning: heat exchanger network energy balance is off by -2.71% (an absolute error greater than 1.00%)
  self._cost()
C:\Users\yrc2\OneDrive\Code\biosteam\biosteam\_unit.py:658: RuntimeWarning: heat exchanger network energy balance

In [11]:
lc.all_metric_mockups[0].units

'$\\mathrm{\\$} \\cdot \\mathrm{ton}^{-1}$'

In [12]:
create_contour_plots(load=True, metric_index=1)

<ipython-input-9-f81380a27ccb>:100: UserWarning: No contour levels were found within the data range.
  CS = plt.contour(100.*X, 100.*Y, data=metric_data, zorder=1e16, linestyles='dashed', linewidths=1.,


array([[[[ 0.746,  0.825],
         [ 0.97 ,  1.073]],

        [[ 1.617,  1.788],
         [ 2.103,  2.325]],

        [[ 2.488,  2.751],
         [ 3.235,  3.576]]],


       [[[ 3.173,  3.507],
         [ 4.124,  4.56 ]],

        [[ 6.874,  7.6  ],
         [ 8.936,  9.88 ]],

        [[10.575, 11.692],
         [13.748, 15.199]]],


       [[[ 5.599,  6.19 ],
         [ 7.278,  8.047]],

        [[12.13 , 13.411],
         [15.77 , 17.434]],

        [[18.662, 20.632],
         [24.261, 26.822]]]])

In [ ]:
create_contour_plots(3, load=True)
plt.suptitle('Biodiesel production from stillage, ethanol production from juice and bagasse (combined)')